# 有监督分类

## 性别鉴定
男性和女性的名字有一些鲜明的特点，以 a, e 和 i 结尾的很可能是女性，而以 k, o, r, s 结尾的很可能是男性，让我们建立一个分类器来更精确地模拟这些差异。
一开始，我们只寻找给定名称的最后一个字母作为特征：

In [1]:
def gender_features(name):
    return {
        'last_letter': name[-1]
    }
gender_features('Shrek')

{'last_letter': 'k'}

接下来，我们使用特征提取器处理名称数据，并划分特征集的结果链表为一个训练集和一个测试集，训练集用于训练一个新的朴素贝叶斯分类器：

In [2]:
import nltk
import random
from nltk.corpus import names

names = ([(name, 'male') for name in names.words('male.txt')] +
        [(name, 'female') for name in names.words('female.txt')])
random.shuffle(names)
featuresets = [(gender_features(name), gender) for (name, gender) in names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print('Neo -> %s' % classifier.classify(gender_features('Neo')))
print('Trinity -> %s' % classifier.classify(gender_features('Trinity')))
print('Accuracy: %f' % nltk.classify.accuracy(classifier, test_set))

Neo -> male
Trinity -> female
Accuracy: 0.750000


最后，我们可以检查分类器，确定哪些特征对于区分名字的性别是最有效的，可以看出以 a 结尾的名字中女性是男性的 387倍，而以 k 结尾的名字中男性是女性的 30 倍，这些比率成为似然比。

In [3]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     34.1 : 1.0
             last_letter = 'k'              male : female =     30.1 : 1.0
             last_letter = 'f'              male : female =     17.5 : 1.0
             last_letter = 'v'              male : female =     11.3 : 1.0
             last_letter = 'p'              male : female =     11.3 : 1.0


在处理大型语料库时，构建一个包含每一个实例特征的链表会使用大量的内存，在这些情况下使用函数 [nltk.classify.apply_features](https://www.nltk.org/_modules/nltk/classify/util.html#apply_features)，返回一个行为像一个链表而不会在内存存储所有特征集的对象：

In [4]:
from nltk.classify import apply_features
train_set = apply_features(gender_features, names[500:])
test_set = apply_features(gender_features, names[:500])

## 选择正确的特征
用于一个给定的学习算法的特征数目是有限的，如果你提供太多的特征，那么该算法将高度依赖训练数据的特性，而一般化到新的例子的效果不会太好。这个问题被称为**过拟合**，当运作在小训练集上尤其会有问题。

In [5]:
def gender_features2(name):
    features = {}
    features['firstletter'] = name[0].lower()
    features['lastletter'] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features['count(%s)' % letter] = name.lower().count(letter)
        features['has(%s)' % letter] = (letter in name.lower())
    return features

featuresets = [(gender_features2(name), gender) for (name, gender) in names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print('Accuracy: %f' % nltk.classify.accuracy(classifier, test_set))

Accuracy: 0.770000


一旦初始特征集被选定，完善特征集的一个非常有效的方法是**错误分析**。首先我们选择一个**开发集**，包含用户创建模型的语料数据，然后将这种开发集分为**训练集**和**开发测试集**。训练集用于训练模型，开发测试集用于进行错误分析，测试集用于系统的最终评估。

In [6]:
train_names, devtest_names, test_names = names[1500:], names[500:1500], names[:500]
train_set = [(gender_features2(name), gender) for (name, gender) in train_names]
devtest_set = [(gender_features2(name), gender) for (name, gender) in devtest_names]
test_set = [(gender_features2(name), gender) for (name, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print('Accuracy: %f' % nltk.classify.accuracy(classifier, devtest_set))

Accuracy: 0.804000


使用开发测试集，我们可以生成一个分类器预测名字性别时的错误列表，检查个别错误按理，尝试确定什么额外信息将使其能够作出正确的决定，然后相应的调整特征集。

In [7]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))
for (tag, guess, name) in sorted(errors)[:5]:
    print('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=male     guess=female   name=Abbey                         
correct=male     guess=female   name=Abbie                         
correct=male     guess=female   name=Adair                         
correct=male     guess=female   name=Adams                         
correct=male     guess=female   name=Agustin                       


浏览错误列表，它明确指出一些多个字母的后缀可以提示名字性别。例如 yn 结尾的名字显示以女性为主，尽管 n 结尾的名字往往是男性；以 ch 结尾的名字通常是男性，尽管以 h 结尾的名字倾向于是女性。因此我们调整特征提取器，包含两个字母后缀的特征：

In [8]:
def gender_features(name):
    return {
        'suffix1': name[-1:],
        'suffix2': name[-2:]
    }
classifier = nltk.NaiveBayesClassifier.train(train_set)
print('Accuracy: %f' % nltk.classify.accuracy(classifier, devtest_set))

Accuracy: 0.804000


不断重复这个错误分析过程，检查存在于由新改进的分类器产生的错误中的模式，每一次错误分析过程被重复，我们应该选择一个不同的开发测试/训练分割，以确保该分类器不会开始反映开发测试集的特质。

## 文档分类
使用文档已经按类别标记的语料库，我们可以建立分类器，自动给新文档添加适当的类别标签。这里我们选择电影评论语料库：

In [9]:
from nltk.corpus import movie_reviews

documents = [(list(movie_reviews.words(fileid)), category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

接下来，我们为文档定义一个特征提取器。对于文档主题识别，我们可以为每个词定义一个特性表示该文档是否包含这个词。为了限制分类器需要处理的特征的数目，我们从整个语料库中选择前 2000 个最频繁的词，然后简单地检查这些词是否在一个给定的文档中。

In [10]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = [w for (w,_) in all_words.most_common(2000)]

def document_features(document):
    document_words = set(document)  # Checking word in a set is much faster than in a list.
    features = {}
    for word in word_features:
        features['contains(%s)' % (word,)] = (word in document_words)
    return features

test_doc_features = document_features(movie_reviews.words('pos/cv957_8737.txt'))
print(dict(list(test_doc_features.items())[:10]))

{'contains(,)': True, 'contains(the)': True, 'contains(.)': True, 'contains(a)': True, 'contains(and)': True, 'contains(of)': True, 'contains(to)': True, "contains(')": True, 'contains(is)': True, 'contains(in)': True}


现在我们用这些特征来训练分类器，为新的电影评论加标签，并使用 show_most_informative_features() 来找出哪些特征是分类器发现最有信息量的。

In [11]:
featuresets = [(document_features(d), c) for (d, c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print('Accuracy: %f' % nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)

Accuracy: 0.810000
Most Informative Features
   contains(outstanding) = True              pos : neg    =     10.7 : 1.0
        contains(seagal) = True              neg : pos    =      8.3 : 1.0
         contains(mulan) = True              pos : neg    =      8.2 : 1.0
   contains(wonderfully) = True              pos : neg    =      6.4 : 1.0
         contains(damon) = True              pos : neg    =      6.0 : 1.0


第5章中，我们建立了一个正则表达式标注器，通过查找词内部的组成，为词选择词性标记。然而，这个正则表达式标注器是手工制作的。作为替代，我们可以训练一个分类器来算出哪个后缀最有信息量。首先，让我们找出最常见的后缀：

In [12]:
from nltk.corpus import brown

suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1
common_suffixes = [w for (w, _) in suffix_fdist.most_common(100)]
print(common_suffixes)

['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


接下来，我们定义一个特征提取函数，检查给定的单词的这些后缀，并训练一个新的决策树分类器：

In [13]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endswith(%s)' % suffix] = word.lower().endswith(suffix)
    return features

tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n, g) in tagged_words]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size: ], featuresets[: size]
classifier = nltk.DecisionTreeClassifier.train(train_set)
print('Accuracy: %f' % nltk.classify.accuracy(classifier, test_set))
classifier.classify(pos_features('cats'))

Accuracy: 0.627051


'NNS'

决策树模型的一个很好的性质是它们往往很容易解释。我们甚至可以指示NLTK将它们以伪代码形式输出:

In [14]:
print(classifier.pseudocode(depth=4))

if endswith(the) == False: 
  if endswith(,) == False: 
    if endswith(s) == False: 
      if endswith(.) == False: return '.'
      if endswith(.) == True: return '.'
    if endswith(s) == True: 
      if endswith(is) == False: return 'PP$'
      if endswith(is) == True: return 'BEZ'
  if endswith(,) == True: return ','
if endswith(the) == True: return 'AT'



## 探索上下文语境
如果特征提取器仅仅看着目标词，我们就没法添加依赖词出现的上下文语境特征。然而上下文语境往往提供关于正确标记的强大线索，例如：标注此 fly，如果知道它前面的词是 a 将使我们能够确定它是一个名词，而不是动词。这里我们修改特征提取器来利用词的上下文特征：

In [15]:
def pos_features(sentence, i):
    features = {
        'suffix(1)': sentence[i][-1:],
        'suffix(2)': sentence[i][-2:],
        'suffix(3)': sentence[i][-3:]
    }
    if i == 0:
        features['prev-word'] = '<START>'
    else:
        features['prev-word'] = sentence[i - 1]
    return features

pos_features(brown.sents()[0], 8)

{'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion', 'prev-word': 'an'}

In [16]:
featuresets = []
for tagged_sent in brown.tagged_sents(categories='news'):
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append((pos_features(untagged_sent, i), tag))
        
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size: ], featuresets[: size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print('Accuracy %f' % nltk.classify.accuracy(classifier, test_set))

Accuracy 0.789160


可以看到，利用上下文特征提高了我们词性标注器的性能。

## 序列分类
一种序列分类器策略，成为**连续分类**或**贪婪序列分类**，是为第一个输入找到最有可能的类标签，然后使用这个问题的答案帮助找到下一个输入的最佳标签，这个过程可以不断重复直到所有的输入都被贴上标签。
在下面的例子中，我们扩展特征提取函数使其具有参数history， 它提供一个我们到目前为止已经为句子预测的标记的链表，history中的每个标记对应句子中的一个词。

In [17]:
def pos_features(sentence, i, history):
    features = {
        'suffix(1)': sentence[i][-1:],
        'suffix(2)': sentence[i][-2:],
        'suffix(3)': sentence[i][-3:]
    }
    if i == 0:
        features['prev-word'] = '<START>'
        features['prev-tag'] = '<START>'
    else:
        features['prev-word'] = sentence[i - 1]
        features['prev-tag'] = history[i - 1]
    return features

class ConsecutivePosTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history)
                train_set.append((featureset, tag))
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
        
    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)
    
tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)
print('Accuracy %f' % tagger.evaluate(test_sents))

Accuracy 0.798053
